[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/analiza-dokumentow/blob/main/16--cleaning.ipynb)

## Znaki specjalne, nadmiarowe spacje, znaczniki HTML, inne jeżeli potrzeba

## Czyszczenie danych
* istotne z perspektywy uwydatnienia poszukiwanych informacji



In [45]:
test_text = '''Termin „post-truth” na tyle zyskał na znaczeniu w opisie rzeczywistości społeczno-politycznej, że w corocznym plebiscycie Oxford Dictionaries redaktorzy uznali go za słowo roku 2016. Argumentowano to m.in. tym, że termin „post-prawda”, używany coraz powszechniej w różnego rodzaju komentarzach politycznych i ważnych publikacjach, przestał mieć znaczenie peryferyjne, stając się terminem samodzielnym, niewymagającym dodatkowego wyjaśniania i definiowania.
'''

In [46]:
test_text.split(".")

['Termin „post-truth” na tyle zyskał na znaczeniu w opisie rzeczywistości społeczno-politycznej, że w corocznym plebiscycie Oxford Dictionaries redaktorzy uznali go za słowo roku 2016',
 ' Argumentowano to m',
 'in',
 ' tym, że termin „post-prawda”, używany coraz powszechniej w różnego rodzaju komentarzach politycznych i ważnych publikacjach, przestał mieć znaczenie peryferyjne, stając się terminem samodzielnym, niewymagającym dodatkowego wyjaśniania i definiowania',
 '\n']

### Nietrywialny przykład z języka polskiego

* prof. Jan Kowalski obiecł zaliczenia w 1-szym terminie
* Koń ciągnie
* Nigdy więcej wojny!
* Ala ma kota a kot ma mleko

zdanie (ang. sentence) vs wypowiedź (ang. utterance)

In [49]:
input_1 = "Ala ma kota a kot ma mleko. Krzyś ma psa, kota, chomika, jaszczurkę i papugę."
input_2 = "Wiedźmin rzucił się w jej stronę, w skoku dobywając miecza."
input_3 = "Stale ucz się. Im więcej różnych rzeczy wiesz, tym lepiej. Czytaj techniczną książkę raz na kwartał. Nietechniczne też czytaj. Uczestnicz w kursach, odwiedzaj konferencje."

## Tokenizacja

podzieleniu analizowanego tekstu na części ``tokeny``

``ala``, ``ma``, ``kota``, ``a``, ``kot``, ``ma``, ``mleko``

In [54]:
#default - space
input_1.split()

['Ala',
 'ma',
 'kota',
 'a',
 'kot',
 'ma',
 'mleko.',
 'Krzyś',
 'ma',
 'psa,',
 'chomika,',
 'jaszczurkę',
 'i',
 'papugę.']

https://www.nltk.org/

In [67]:
pip install nltk


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
import nltk


In [70]:
from nltk.tokenize import word_tokenize

word_tokenize(input_2)

['Wiedźmin',
 'rzucił',
 'się',
 'w',
 'jej',
 'stronę',
 ',',
 'w',
 'skoku',
 'dobywając',
 'miecza',
 '.']

## Uni-Gram | Bi-Gram | Tri-Gram | N-Gram

Ala ma kota a kot ma mleko

```
Uni-Gram (1-gram) Ala, ma, kota, a, kot, ma, mleko
Bi-Gram (2-gram) Ala ma, ma kota, kota a, a kot, kot ma, ma mleko 
Tri-Gram (3-gram) Ala ma kota, ma kota a, kota a kot,  a kot ma, kot ma mleko 
...
```


In [78]:
from nltk import ngrams
list(ngrams(word_tokenize(input_1), 3))

[('Ala', 'ma', 'kota'),
 ('ma', 'kota', 'a'),
 ('kota', 'a', 'kot'),
 ('a', 'kot', 'ma'),
 ('kot', 'ma', 'mleko'),
 ('ma', 'mleko', '.'),
 ('mleko', '.', 'Krzyś'),
 ('.', 'Krzyś', 'ma'),
 ('Krzyś', 'ma', 'psa'),
 ('ma', 'psa', ','),
 ('psa', ',', 'chomika'),
 (',', 'chomika', ','),
 ('chomika', ',', 'jaszczurkę'),
 (',', 'jaszczurkę', 'i'),
 ('jaszczurkę', 'i', 'papugę'),
 ('i', 'papugę', '.')]

In [79]:
from nltk import sent_tokenize
sent_tokenize(input_1)

['Ala ma kota a kot ma mleko.', 'Krzyś ma psa, chomika, jaszczurkę i papugę.']

In [81]:
sent_tokenize('prof. Jan Kowalski obiecł zaliczenia w 1-szym terminie')

['prof. Jan Kowalski obiecł zaliczenia w 1-szym terminie']

## Normalizacja

Upraszczanie do wspólnej wartości. 

```
kot -> kot
kota -> kot
kotem -> kot
jeden -> jeden
jedna -> jeden
jeden -> 1
```

## Stemming
bazując na definicji z angielskiej wikipedii jest to proces polegający na wydobyciu z wybranego wyrazu tzw. rdzenia, a więc tej jego części, która jest odporna na odmiany przez przyimki, rodzaje itp.

```
Wiedźmin -> Wiedźmin
rzucił -> rzucił
się -> się 
w -> w 
jej -> je
stronę -> stron
w -> w 
skoku -> skok
dobywając -> dobywa
miecza -> miecz
```

## Lematyzacja

pojęcie to jest bardzo podobne do powyższego, a oznacza sprowadzenie grupy wyrazów stanowiących odmianę danego zwrotu do wspólnej postaci, umożliwiającej traktowanie ich wszystkich jako te samo słowo.

```
Wiedźmin -> Wiedźmin
rzucił -> rzuca
się -> się 
w -> w 
jej -> jej
stronę -> strona
w -> w 
skoku -> skok
dobywając -> dobywa
miecza -> miecz
```

## Oczyszczanie | Przekształcanie 

In [82]:
pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [83]:
import pandas as pd

BOOKS = pd.read_pickle('var/books.pkl')

In [161]:
BOOKS['content'][6][:255]

'Adam Mickiewicz\r\n\r\nDziady. Widowisko, część I\r\n\r\n\r\n\r\n/ Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo, fortepiano, okno z lewej strony w pole; na prawej wielkie zwierciadło; świeca gasnąca na stole i księga rozłożona (romans Vale'

In [171]:
BOOKS['content'][6][-1800:-1]

'ą\r\nPewna istota, która z oczu cię [nie] traci,\r\nI że chce ciebie w ludzkiej nawiedzić postaci,\r\nJeżeli to, coś przyrzekł, zachowasz niezłomnie…\r\n\r\n\r\nGUSTAW\r\n\r\nPrzebóg! co to ma znaczyć?… Nie zbliżaj się do mnie!\r\n\r\nI na tym się rękopis kończy\r\n\r\n\r\n\r\n\r\n-----\r\nTa lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.\r\nWersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.\r\n\r\nUtwór opracowany został w ramach projektu Wolne Lektury przez fundację Nowoczesna Polska.\r\n\r\nWszystkie zasoby Wolnych Lektur możesz swobodnie wykorzystywać, publikować i rozpowszechniać pod warunkiem zachowania warunków licencji i zgodnie z Zasadami wykorzystania Wolnych Lektur.\r\nTen utwór jest w domenie publicznej.\r\nWszystkie materiały dodatkowe (przypisy, motywy literackie) są udostępnione na Licencji Wolnej Sztuki 1.3: https://

https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt

```
-----
Ta lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.
Wersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.

Utwór opracowany został w ramach projektu Wolne Lektury przez fundację Nowoczesna Polska.
```

In [198]:
test_text = '''
Adam Mickiewicz\r\n\r\nDziady.

ISBN 978-83-288-2972-5
Widowisko, część I\r\n\r\n\r\n\r\n/ Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo, fortepiano, okno z lewej strony w pole; na prawej wielkie zwierciadło; świeca gasnąca na stole i księga rozłożona (romans Vale
----- Ta lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.
Wersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.

Utwór opracowany został w ramach projektu Wolne Lektury przez fundację Nowoczesna Polska.
'''

In [208]:
import re
def preprocess_text(text):
    text = re.sub(r"^.*\n","", text) 
    text = re.sub(r"ISBN.*[1-9\-]{16,20}","", text, flags=re.IGNORECASE) 
    
    text = re.sub(u"[ \n]+", " ", text) # newlines -> spaces
    text = re.sub(u"[ \r]+", " ", text) # \r -> spaces
    text = text.strip()
    text = re.sub(r"----- Ta lektura.*","", text)


    return text


assert ('\n' not in preprocess_text(test_text))
assert ('\r' not in preprocess_text(test_text))
assert ('  ' not in preprocess_text(test_text))
assert ('Ta lektura,' not in preprocess_text(test_text))
assert ('ISBN' not in preprocess_text(test_text))
assert ('2972-5' not in preprocess_text(test_text))


In [204]:
test_text = preprocess_text(test_text)

In [205]:
test_text

'Adam Mickiewicz Dziady. Widowisko, część I / Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo, fortepiano, okno z lewej strony w pole; na prawej wielkie zwierciadło; świeca gasnąca na stole i księga rozłożona (romans Vale '

In [177]:
def split_to_sentences(text):
  return [re.sub(r"^ ","",l) for l in re.split('\.|,|\?|!|:', text)]

assert(['hello world', "Hello John"] == split_to_sentences("hello world! Hello John"))

### interpunkcja

In [178]:
import string 
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

In [179]:
assert(remove_punct("hello, world!") == "hello world")

In [180]:
remove_punct(test_text)

'Adam Mickiewicz Dziady Widowisko część I  Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo fortepiano okno z lewej strony w pole na prawej wielkie zwierciadło świeca gasnąca na stole i księga rozłożona romans Vale '

### Stop words

In [181]:
pip install requests


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [182]:
import requests

In [183]:
stop_words = (requests
         .get('https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt')
         .text
         .split('\n'))

In [184]:

stop_words[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

In [185]:
def remove_stop_words(text):
  return " ".join([word for word in text.split(' ') if word not in stop_words])

assert "Cześć czołem" == remove_stop_words("Cześć i czołem")

In [186]:
BOOKS.head()

,author,filename,content
0,Zeromski,zeromski-oko-za-oko.txt,Stefan Żeromski\r\n\r\nOko za oko\r\n\r\nISBN ...
1,Zeromski,wszystko-i-nic.txt,Stefan Żeromski\r\n\r\nWszystko i nic\r\n\r\nI...
2,Zeromski,zeromski-rozdziobia-nas-kruki-wrony.txt,"Stefan Żeromski\r\n\r\nRozdzióbią nas kruki, w..."
3,Zeromski,silaczka.txt,Stefan Żeromski\r\n\r\nSiłaczka\r\n\r\nISBN 97...
4,Zeromski,syzyfowe-prace.txt,Stefan Żeromski\r\n\r\nSyzyfowe prace\r\n\r\nI...


In [209]:
BOOKS['content_txt'] = (BOOKS.content
                             .apply(preprocess_text)
                             .apply(lambda s: s.lower())
                             .apply(remove_stop_words)
                             .apply(sent_tokenize)
                            )

In [222]:
def clean_sentences(sentence):
    sentence = remove_punct(sentence)
    sentence.replace("—", "")
    
    return sentence

In [223]:
BOOKS['sentences_clean'] = BOOKS.content_txt.apply(lambda sentences: list(map(clean_sentences, sentences)))

In [226]:
BOOKS[:4]

,author,filename,content,content_txt,sentences_clean
0,Zeromski,zeromski-oko-za-oko.txt,Stefan Żeromski\r\n\r\nOko za oko\r\n\r\nISBN ...,[oko oko zawiadowca stacji trebizondów wielki ...,[oko oko zawiadowca stacji trebizondów wielki ...
1,Zeromski,wszystko-i-nic.txt,Stefan Żeromski\r\n\r\nWszystko i nic\r\n\r\nI...,"[ledwie drzwi zamknęły olbromscy, — rafał syn ...",[ledwie drzwi zamknęły olbromscy — rafał syn j...
2,Zeromski,zeromski-rozdziobia-nas-kruki-wrony.txt,"Stefan Żeromski\r\n\r\nRozdzióbią nas kruki, w...","[rozdzióbią kruki, wrony… żywy promień zdołał ...",[rozdzióbią kruki wrony… żywy promień zdołał p...
3,Zeromski,silaczka.txt,Stefan Żeromski\r\n\r\nSiłaczka\r\n\r\nISBN 97...,[siłaczka nienajlepszym humorze powrócił domu ...,[siłaczka nienajlepszym humorze powrócił domu ...


In [227]:
BOOK_LINES = BOOKS[['author',  'sentences_clean']].explode('sentences_clean')

In [228]:
BOOK_LINES = BOOK_LINES.reset_index(drop=True)

In [229]:
BOOK_LINES.head()

,author,sentences_clean
0,Zeromski,oko oko zawiadowca stacji trebizondów wielki w...
1,Zeromski,zgromadzeni dokoła stołu przedstawiciele pewne...
2,Zeromski,urzędnicy powiatowi wtłoczeni małą kanapę zasu...
3,Zeromski,geometra drugiej klasy pięścią oczyma wzniesio...
4,Zeromski,młody doktor miejscowy czarny chudy sztalugi ż...


In [230]:
BOOK_LINES.groupby('author').count()

,sentences_clean
author,
Mickiewicz,4678
Orzeszkowa,21437
Prus,25648
Reymont,23197
Sienkiewicz,38191
Zeromski,6311


Duża różnica w elementach danej klasy!! Może mieć wpływ na rezultaty

In [231]:
BOOK_LINES['words'] = BOOK_LINES.sentences_clean.apply(lambda s: len(s.split()))

In [232]:
BOOK_LINES.groupby('author')['words'].describe()


,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,4678.0,11.845019,10.179747,1.0,4.0,9.0,16.0,95.0
Orzeszkowa,21437.0,12.990624,11.369117,1.0,4.0,10.0,18.0,146.0
Prus,25648.0,9.680170,7.651949,1.0,4.0,8.0,12.0,107.0
Reymont,23197.0,11.121179,12.415336,1.0,4.0,7.0,14.0,209.0
Sienkiewicz,38191.0,9.333613,7.722366,1.0,4.0,7.0,13.0,123.0
Zeromski,6311.0,10.743622,7.873517,1.0,5.0,9.0,14.0,70.0


In [233]:
BOOK_LINES = BOOK_LINES[BOOK_LINES['words'] != 0]

In [234]:
BOOK_LINES.groupby('author')['words'].quantile(0.98)

author
Mickiewicz     40.0
Orzeszkowa     44.0
Prus           32.0
Reymont        48.0
Sienkiewicz    31.0
Zeromski       33.0
Name: words, dtype: float64

In [235]:
BOOK_LINES.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,4678.0,11.845019,10.179747,1.0,4.0,9.0,16.0,95.0
Orzeszkowa,21437.0,12.990624,11.369117,1.0,4.0,10.0,18.0,146.0
Prus,25648.0,9.680170,7.651949,1.0,4.0,8.0,12.0,107.0
Reymont,23197.0,11.121179,12.415336,1.0,4.0,7.0,14.0,209.0
Sienkiewicz,38191.0,9.333613,7.722366,1.0,4.0,7.0,13.0,123.0
Zeromski,6311.0,10.743622,7.873517,1.0,5.0,9.0,14.0,70.0


In [238]:
words_together = " ".join(BOOK_LINES[BOOK_LINES.author == 'Mickiewicz'].sentences_clean)

In [39]:
pip install wordcloud

  Using cached wordcloud-1.8.2.2.tar.gz (220 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-13-x86_64-cpython-311
      creating build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/wordcloud_cli.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/_version.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/__init__.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/tokenization.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/wordcloud.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/color_from_image.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying

In [37]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width = 400, height = 400, 
                      background_color = 'black', 
                      min_font_size = 5,
                      stop_words=stop_words,
                      max_words=1000,
                      collocations=False).generate("Ala ma kota a kot ma mleko")

plt.figure(figsize = (12, 12), facecolor = 'lavender')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 2) 
plt.show()


ModuleNotFoundError: No module named 'wordcloud'